In [1]:
import sys
sys.path.append("../")
from libraries.dataManipulation import *
from libraries.gridManipulation import *
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt
import math
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from sociophysicsDataHandler import SociophysicsDataHandler

In [3]:
def convert_to_unixtimestamp(datetime_utc):
    return (datetime_utc - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [4]:
#Initialize variables
time = pd.Timestamp('2020-04-20T12:00:00') #Starting time of simulation
sim_duration = 100 #How long the simulation lasts in seconds

train_present = False

#Number of zones
zone_rows = 4
zone_columns = 18
n_zones = zone_rows * zone_columns

#Information about people
current_ID = 1000 #People will also have an ID in the simulation, only a different one
offboarding_people_locs = {} #Dictionary with person IDs as keys and zone IDs as values
onboarding_people_locs = {} #Dictionary with person IDs as keys and zone IDs as values
density = {} #Dictionary with zone IDs as keys and density as values
#Set all densities to 0
for i in range(n_zones):
    density[i] = 0

#Variables that have to be determined in different steps
markov_matrix_offboarding = None
markov_matrix_onboarding_no_train = None
markov_matrix_onboarding_train = None
density_influence_table = None
people_in_dictionary_off = {time:[1]}
people_in_dictionary_on = {time:[2]}

In [5]:
#Check for train
#TODO
def check_train(timestamp):
    train_present = False

In [6]:
#Spawn people
def spawn_people(timestamp):
    global current_ID
    if timestamp in people_in_dictionary_off:
        spawn_zones = people_in_dictionary_off[timestamp]
        for zone in spawn_zones:
            offboarding_people_locs[current_ID] = zone
            current_ID += 1
    
    if timestamp in people_in_dictionary_on:
        spawn_zones = people_in_dictionary_on[timestamp]
        for zone in spawn_zones:
            onboarding_people_locs[current_ID] = zone
            current_ID += 1

In [7]:
#Determine the density based on the locations of people
def calculate_density():
    locs = {**onboarding_people_locs,**offboarding_people_locs}
    
    #Create empty density dictionary
    density = {} #Dictionary with zone IDs as keys and density as values
    #Set all densities to 0
    for i in range(n_zones):
        density[i] = 0
    
    #Cycle through all the people in the locs dictionary
    for i in locs.keys():
        density[locs[i]] += 1
        
    return density

In [8]:
#Get a new zone for a person a markov matrix and a randomly generated number
def get_new_zone(rand, current_zone, markov_matrix):
    
    #Get the probabilities for the zone and apply the randomness
    probabilities = markov_matrix.iloc[current_zone].to_numpy()
    i = 0 #Iterator to keep track of which column/movement must be selected
    while rand > probabilities[i]:
        rand -= probabilities[i]
        i += 1
    
    #Get the new zone based on the movement (column of markov matrix) that is selected
    if i == 0: return current_zone
    if i == 1: return current_zone - zone_rows - 1
    if i == 2: return current_zone - zone_rows
    if i == 3: return current_zone - zone_rows + 1
    if i == 4: return current_zone - 1
    if i == 5: return current_zone + 1
    if i == 6: return current_zone + zone_rows - 1
    if i == 7: return current_zone + zone_rows
    if i == 8: return current_zone + zone_rows + 1
    if i == 9: return -1
    else: return -1
    

In [9]:
#Adjust the probabilities of moving between zones according to density in neighbouring zones
#TODO
def adjust_markov_matrix(matrix, density):
    return matrix

In [10]:
#Change the position of people according to the corresponding markov matrix
def move():
    rand = random.random()

    #Loop through offboarders with markov matrix adjusted for density
    markov_matrix = adjust_markov_matrix(markov_matrix_offboarding, density)
    
    for offboarder_ID in offboarding_people_locs.keys():
        
        current_zone = offboarding_people_locs[offboarder_ID]
        
        new_zone = get_new_zone(rand, current_zone, markov_matrix)
        
        offboarding_people_locs[offboarder_ID] = new_zone
    
    #Loop through onboarders with markov matrix adjusted for density and whether there is a train
    if trainpresent:
        markov_matrix = adjust_markov_matrix(markov_matrix_onboarding_train, density)
    else:
        markov_matrix = adjust_markov_matrix(markov_matrix_onboarding_no_train, density)
        
    for onboarder_ID in onboarding_people_locs.keys():
        
        current_zone = onboarding_people_locs[onboarder_ID]
        
        new_zone = get_new_zone(rand, current_zone, markov_matrix)

        onboarding_people_locs[onboarder_ID] = new_zone
            

In [11]:
# SIMULATE

for i in range(sim_duration):
    
    spawn_people(time)
    
    check_train(time)
    
    move()
    
    density = calculate_density()
    
    time += 1
    

AttributeError: 'NoneType' object has no attribute 'iloc'